In [1]:
import nltk
from nltk import word_tokenize
import string

In [2]:
#nltk.download('punkt')

In [3]:
Text = "The user wants to upload the data"

In [4]:
Key_prhases = []
words = []

In [5]:
import nltk
from nltk import word_tokenize
import string

#nltk.download('punkt')

def clean(text):
    text = text.lower()
    printable = set(string.printable)
    text = filter(lambda x: x in printable, text)
    text = "".join(list(text))
    return text

Cleaned_text = clean(Text)
# print(Cleaned_text)
text = word_tokenize(Cleaned_text)

print ("Tokenized Text: \n")
print (text)

Tokenized Text: 

['the', 'user', 'wants', 'to', 'upload', 'the', 'data']


In [6]:
#nltk.download('averaged_perceptron_tagger')
  
POS_tag = nltk.pos_tag(text)

print ("Tokenized Text with POS tags: \n")
print (POS_tag)

Tokenized Text with POS tags: 

[('the', 'DT'), ('user', 'NN'), ('wants', 'VBZ'), ('to', 'TO'), ('upload', 'VB'), ('the', 'DT'), ('data', 'NNS')]


In [7]:
#nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

adjective_tags = ['JJ','JJR','JJS']

lemmatized_text = []

for word in POS_tag:
    if word[1] in adjective_tags:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos="a")))
    else:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0]))) #default POS = noun
        
print ("Text tokens after lemmatization of adjectives and nouns: \n")
print (lemmatized_text)

Text tokens after lemmatization of adjectives and nouns: 

['the', 'user', 'want', 'to', 'upload', 'the', 'data']


In [8]:
POS_tag = nltk.pos_tag(lemmatized_text)

print ("Lemmatized text with POS tags: \n")
print (POS_tag)

Lemmatized text with POS tags: 

[('the', 'DT'), ('user', 'NN'), ('want', 'VBP'), ('to', 'TO'), ('upload', 'VB'), ('the', 'DT'), ('data', 'NNS')]


In [9]:
stopwords = []

wanted_POS = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','FW'] 

for word in POS_tag:
    if word[1] not in wanted_POS:
        stopwords.append(word[0])

punctuations = list(str(string.punctuation))

stopwords = stopwords + punctuations

In [10]:
stopword_file = open("long_stopwords.txt", "r")
#Source = https://www.ranks.nl/stopwords

lots_of_stopwords = []

for line in stopword_file.readlines():
    lots_of_stopwords.append(str(line.strip()))

stopwords_plus = []
stopwords_plus = stopwords + lots_of_stopwords
stopwords_plus = set(stopwords_plus)

#Stopwords_plus contain total set of all stopwords

In [11]:
processed_text = []
for word in lemmatized_text:
    if word not in stopwords_plus:
        processed_text.append(word)
print (processed_text)

['user', 'data']


In [12]:
vocabulary = list(set(processed_text))
print (vocabulary)

['data', 'user']


In [13]:
import numpy as np
import math
vocab_len = len(vocabulary)

weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

score = np.zeros((vocab_len),dtype=np.float32)
window_size = 3
covered_coocurrences = []

for i in range(0,vocab_len):
    score[i]=1
    for j in range(0,vocab_len):
        if j==i:
            weighted_edge[i][j]=0
        else:
            for window_start in range(0,(len(processed_text)-window_size)):
                
                window_end = window_start+window_size
                
                window = processed_text[window_start:window_end]
                
                if (vocabulary[i] in window) and (vocabulary[j] in window):
                    
                    index_of_i = window_start + window.index(vocabulary[i])
                    index_of_j = window_start + window.index(vocabulary[j])
                    
                    # index_of_x is the absolute position of the xth term in the window 
                    # (counting from 0) 
                    # in the processed_text
                      
                    if [index_of_i,index_of_j] not in covered_coocurrences:
                        weighted_edge[i][j]+=1/math.fabs(index_of_i-index_of_j)
                        covered_coocurrences.append([index_of_i,index_of_j])

In [14]:
inout = np.zeros((vocab_len),dtype=np.float32)

for i in range(0,vocab_len):
    for j in range(0,vocab_len):
        inout[i]+=weighted_edge[i][j]

In [15]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(0,vocab_len):
        
        summation = 0
        for j in range(0,vocab_len):
            if weighted_edge[i][j] != 0:
                summation += (weighted_edge[i][j]/inout[j])*score[j]
                
        score[i] = (1-d) + d*(summation)
    
    if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
        print("Converging at iteration "+str(iter)+"....")
        break

Converging at iteration 1....


In [16]:
for i in range(0,vocab_len):
    print("Score of "+vocabulary[i]+": "+str(score[i]))

Score of data: 0.15
Score of user: 0.15


In [17]:
phrases = []

phrase = " "
for word in lemmatized_text:
    
    if word in stopwords_plus:
        if phrase!= " ":
            phrases.append(str(phrase).strip().split())
        phrase = " "
    elif word not in stopwords_plus:
        phrase+=str(word)
        phrase+=" "

print("Partitioned Phrases (Candidate Keyphrases): \n")
print(phrases)

Partitioned Phrases (Candidate Keyphrases): 

[['user']]


In [18]:
unique_phrases = []

for phrase in phrases:
    if phrase not in unique_phrases:
        unique_phrases.append(phrase)

print("Unique Phrases (Candidate Keyphrases): \n")
print(unique_phrases)

Unique Phrases (Candidate Keyphrases): 

[['user']]


In [19]:
for word in vocabulary:
    #print word
    for phrase in unique_phrases:
        if (word in phrase) and ([word] in unique_phrases) and (len(phrase)>1):
            #if len(phrase)>1 then the current phrase is multi-worded.
            #if the word in vocabulary is present in unique_phrases as a single-word-phrase
            # and at the same time present as a word within a multi-worded phrase,
            # then I will remove the single-word-phrase from the list.
            unique_phrases.remove([word])
            
print("Thinned Unique Phrases (Candidate Keyphrases): \n")
print(unique_phrases)

Thinned Unique Phrases (Candidate Keyphrases): 

[['user']]


In [20]:
phrase_scores = []
keywords = []
for phrase in unique_phrases:
    phrase_score=0
    keyword = ''
    for word in phrase:
        keyword += str(word)
        keyword += " "
        phrase_score+=score[vocabulary.index(word)]
    phrase_scores.append(phrase_score)
    keywords.append(keyword.strip())

i=0
for keyword in keywords:
    print ("Keyword: '"+str(keyword)+"', Score: "+str(phrase_scores[i]))
    i+=1

Keyword: 'user', Score: 0.15000000596046448


In [21]:
Keyword_column = set()
sorted_index = np.flip(np.argsort(phrase_scores),0)

keywords_num = len(keywords)

for i in range(0,keywords_num):
    Keyword_column.add(keywords[sorted_index[i]])

In [22]:
Keyword_column

{'user'}

In [23]:
sentence=nltk.pos_tag(text)
patterns= r"""NPR: {<TO.?>*<VB>+<VBN.*>*<PDT.*>*<DT.*>*<IN.*>*<DT.*>*<JJ.*>*<NN.*>}
             NNR:{<NN.*>*<NN.*>}"""
chunker = nltk.RegexpParser(patterns)
output =chunker.parse(sentence)
for tree in output.subtrees(filter=lambda t: t.label() == 'NPR'):
    Keyword_column.add(' '.join([child[0] for child in tree.leaves()]))
#     Files_column.append(files[i])
#     type_col.append('NPR')
for tree in output.subtrees(filter=lambda t: t.label() == 'NNR'):
    Keyword_column.add(' '.join([child[0] for child in tree.leaves()]))
#     Files_column.append(files[i])
#     type_col.append('NNR')      

In [24]:
Keyword_column

{'to upload the data', 'user'}

In [26]:
Keyword_column = list(Keyword_column)

In [40]:
for i in range(0,len(Keyword_column)):
    Key_prhases.append(Text)
    words.append(Keyword_column[i])

In [41]:
Key_prhases

['The user wants to upload the data', 'The user wants to upload the data']

In [42]:
words

['to upload the data', 'user']